In [8]:
import pickle
import IDRTools
import spectral_lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact

In [9]:
SG_PATH = '/Users/samdixon/data/BLACKSTON_spec_feat_py3.pkl'
with open(SG_PATH, 'rb') as f:
    SG = pickle.load(f)
DS = IDRTools.Dataset(subset=['training', 'validation'],
                      data_dir='/Users/samdixon/data/BLACKSTONE')
V_LINE_NAME_MAP = {'SiII4000': 'SiII_4128',
                   'SIIW_L': 'SiII_5454',
                   'SIIW_R': 'SiII_5640',
                   'SiII5972': 'SiII_5972',
                   'SiII6355': 'SiII_6355'}

In [10]:
sn = np.random.choice(DS.sne)
spec = sn.spec_nearest()
spl = spectral_lines.Spl(spec)
sg_data = SG[sn.target_name]['spectra'][spec.obs_exp]

In [18]:
def plot_near_max_spec(sn, norm, line):
    spec = sn.spec_nearest()
    wave, flux, var = spec.rf_spec()
    spl = spectral_lines.Spl(spec, norm=norm, line=line)
    sg_data = SG[sn.target_name]['spectra'][spec.obs_exp]
    
    plt.plot(spl.wave_feat, spl.flux_feat)
    smooth_wave, smooth_flux = spl.get_smoothed_feature_spec()
    plt.plot(smooth_wave, smooth_flux)
    plt.axvspan(*spectral_lines.extrema_lims[line]['b'], alpha=0.2)
    plt.axvspan(*spectral_lines.extrema_lims[line]['r'], color='C3', alpha=0.2)
    try:
        plt.axvspan(*spectral_lines.vel_lims[line], alpha=0.2, color='k')
    except KeyError:
        pass
    plt.axvline(spl.minimum, color='k')
    plt.axvline(spl.maxima['blue_max_wave'], color='C0')
    plt.axvline(spl.maxima['red_max_wave'], color='C3')
    for i, _ in enumerate(sg_data['phrenology.lbd_EW{}'.format(line)]):
        if i==1:
            plt.axvline(_, linestyle='--', color='C3')
        else:
            plt.axvline(_, linestyle='--', color='C0')
    try:
        plt.axvline(sg_data['phrenology.v{}_lbd'.format(V_LINE_NAME_MAP[line])], linestyle='--', color='k')
    except KeyError:
        pass
    plt.title('{} {:+0.3f} days'.format(sn.target_name, spec.salt2_phase))
    plt.xlabel('Rest-frame wavelength ($\AA$)')
    plt.ylabel('Norm. flux (A.U.)')
#     plt.ylim(0, 2)

In [19]:
interact(plot_near_max_spec, sn=sorted([sn for sn in DS.sne if np.abs(sn.spec_nearest().salt2_phase)<2.5],
                                       key=lambda x: x.target_name),
         norm=['SNID', 'line'], line=spectral_lines.line_names)

interactive(children=(Dropdown(description='sn', options=(CSS110918_01, CSS110918_02, CSS120424_01, CSS130502_…

<function __main__.plot_near_max_spec(sn, norm, line)>